# Quantum Appromaxiate Optimization Algorithm Tutorial

## Introduction
QAOA is a kind of quantum algorithm which can solve combinatorial optimization problem, for example, optimal partition problem. Here we show an example of QAOA.

In [1]:
from pyqpanda.Hamiltonian import PauliOperator
from pyqpanda.Hamiltonian.QubitOperator import *
from pyqpanda import *
from pyqpanda.utils import *
from pyqpanda.Algorithm.hamiltonian_simulation import *

## Problem Description
First, we construct a graph  *G* which represents the optimal problem, codes *[i,j,C<sub>ij]* means the weight of edge *G<sub>ij* is *w*. The problem is finding a partition which divides *G* into $ G_1 $ and $ G_2 $ and makes the *SUM* is maximal,where *SUM* is defined as:
$$ SUM= \sum_{i\in G_1,j\in G_2} C_{ij} $$  

In [2]:
#define graph
graph=[[0,5,0.18],[0,6,0.49],[1,6,0.59],[1,7,0.44],\
[2,7,0.56],[2,8,0.63],[4,9,0.43],[5,10,0.23],[6,11,0.64],\
[7,12,0.60],[8,13,0.36],[9,14,0.52],[10,15,0.40],[10,16,0.41],\
[11,16,0.57],[11,17,0.50],[12,17,0.71],[12,18,0.40],[13,18,0.72],\
[13,3,0.81],[14,3,0.29]]

## Target Hamiltonian

$$ H_c=-\frac{1}{2}\sum_{ij}C_{ij}(1-\sigma_i^z\sigma_j^z)  $$ 
Construct quantum circuit of $ e^{-i\gamma H_c} $

In [3]:
#decide qubit number
qn_=get_qn_from_graph(graph)
init()
prog=QProg()
q=qAlloc_many(qn_)
prog.insert(ising_model(q,graph,0.1))
print(to_qrunes(prog))

#QUBITS_NUM 19
#CREGS_NUM 0
CNOT 0,5
RZ 5,"0.036000"
CNOT 0,5
CNOT 0,6
RZ 6,"0.098000"
CNOT 0,6
CNOT 1,6
RZ 6,"0.118000"
CNOT 1,6
CNOT 1,7
RZ 7,"0.088000"
CNOT 1,7
CNOT 2,7
RZ 7,"0.112000"
CNOT 2,7
CNOT 2,8
RZ 8,"0.126000"
CNOT 2,8
CNOT 4,9
RZ 9,"0.086000"
CNOT 4,9
CNOT 5,10
RZ 10,"0.046000"
CNOT 5,10
CNOT 6,11
RZ 11,"0.128000"
CNOT 6,11
CNOT 7,12
RZ 12,"0.120000"
CNOT 7,12
CNOT 8,13
RZ 13,"0.072000"
CNOT 8,13
CNOT 9,14
RZ 14,"0.104000"
CNOT 9,14
CNOT 10,15
RZ 15,"0.080000"
CNOT 10,15
CNOT 10,16
RZ 16,"0.082000"
CNOT 10,16
CNOT 11,16
RZ 16,"0.114000"
CNOT 11,16
CNOT 11,17
RZ 17,"0.100000"
CNOT 11,17
CNOT 12,17
RZ 17,"0.142000"
CNOT 12,17
CNOT 12,18
RZ 18,"0.080000"
CNOT 12,18
CNOT 13,18
RZ 18,"0.144000"
CNOT 13,18
CNOT 13,3
RZ 3,"0.162000"
CNOT 13,3
CNOT 14,3
RZ 3,"0.058000"
CNOT 14,3



## Driver Hamiltonian $$ H_d=\sum_{i}\sigma_i^x  $$ 
construct quantum circuit of $$ e^{-i\beta H_d} $$

In [4]:
progx=QProg()
beta=0.5
progx.insert(pauliX_model(q,beta))
print(to_qrunes(progx))

#QUBITS_NUM 19
#CREGS_NUM 0
RX 0,"1.000000"
RX 1,"1.000000"
RX 2,"1.000000"
RX 3,"1.000000"
RX 4,"1.000000"
RX 5,"1.000000"
RX 6,"1.000000"
RX 7,"1.000000"
RX 8,"1.000000"
RX 9,"1.000000"
RX 10,"1.000000"
RX 11,"1.000000"
RX 12,"1.000000"
RX 13,"1.000000"
RX 14,"1.000000"
RX 15,"1.000000"
RX 16,"1.000000"
RX 17,"1.000000"
RX 18,"1.000000"



## QAOA quantum program generation

In [5]:
gamma=[0.5,-0.5]
beta=[0.5,-0.5]
result=quantum_approximate_optimization_algorithm(
    graph,
    gamma,
    beta,
    use_prob_run=True,
    use_quick_measure=True,
    multiProcessing=False,    
    shots_=100, 
    dataType="list")
print(result)

-8.64


## QAOA quantum program execution
parameter optimization of QAOA and print the optimized parameter.
result.x is sequence of optimized parameter
qaoa.txt:save optimization process data

In [6]:
step=2
f=open("qaoa.txt", 'w')
f.close()
gamma_=[]
beta_=[]
shots=1
for i in range(step):
    gamma_.append(0.5)
    beta_.append(-0.5)
initial_guess=gamma_+beta_
print(initial_guess,'initial guess')
#Scipy.optimize.minize function as optimize funciton
result = minimize(binding(graph,shots), initial_guess, method="Powell")
print(result)

[0.5, 0.5, -0.5, -0.5] initial guess
   direc: array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])
     fun: array(-7.27)
 message: 'Optimization terminated successfully.'
    nfev: 63
     nit: 1
  status: 0
 success: True
       x: array([ 1.41881793,  0.47680084, -0.25510086, -4.01263051])


## *QAOA function*
All funciton described above can realized by *"qaoa"* function

In [10]:
result=qaoa(graph=graph,step_=2,shots_=100, method="Powell")
print(result)

   direc: array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])
     fun: array(-10.48)
 message: 'Optimization terminated successfully.'
    nfev: 115
     nit: 2
  status: 0
 success: True
       x: array([ 1.31849533,  3.84878686, -0.68766259,  0.33267957])
